#code


In [ ]:
import os
os.environ['kaggle_config_dir']='/content'
! kaggle datasets download -d nelgiriyewithana/emotions
! unzip /content/emotions.zip

Dataset URL: https://www.kaggle.com/datasets/nelgiriyewithana/emotions
License(s): other
 32% 5.00M/15.7M [00:00<00:00, 27.6MB/s]
100% 15.7M/15.7M [00:00<00:00, 75.1MB/s]
Archive:  /content/emotions.zip
  inflating: text.csv                


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as ps
import warnings
warnings.filterwarnings('ignore')
import numpy as np
df=pd.read_csv('/content/text.csv')

In [ ]:
rating_to_emotion = {
    5: 'surprise',
    4: 'fear',
    3: 'angry',
    2: 'lovely',
    1: 'joy',
    0: 'sad'
}

# Map the numerical ratings to categorical emotions

df['label'] = df['label'].map(rating_to_emotion)
df=df.drop(columns='Unnamed: 0')
df=df.iloc[:200000]

In [ ]:
!pip install transformers tensorflow


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification
from sklearn.preprocessing import LabelEncoder

# Load and preprocess your dataset
# Assuming df is already defined and contains the text and label columns

# Encode labels
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])

# Split the dataset into train and test sets
train_texts, test_texts, train_labels, test_labels = train_test_split(df['text'], df['label'], test_size=0.2, random_state=42)

# Initialize DistilBERT tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=len(label_encoder.classes_))

# Tokenize texts
train_encodings = tokenizer(list(train_texts.values), truncation=True, padding=True)
test_encodings = tokenizer(list(test_texts.values), truncation=True, padding=True)

# Convert encodings to TensorFlow dataset format
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
))

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    test_labels
))

# Optimize the dataset pipeline
AUTOTUNE = tf.data.AUTOTUNE

# Function to prepare dataset
def prepare_dataset(dataset, batch_size, shuffle=False, cache=True):
    if cache:
        dataset = dataset.cache()
    if shuffle:
        dataset = dataset.shuffle(buffer_size=10000)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(buffer_size=AUTOTUNE)
    return dataset

# Prepare train and test datasets
train_dataset = prepare_dataset(train_dataset, batch_size=32, shuffle=True)
test_dataset = prepare_dataset(test_dataset, batch_size=64)



tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

# Train the model
history = model.fit(train_dataset, epochs=3, validation_data=test_dataset)

# Evaluate the model
eval_result = model.evaluate(test_dataset)

# Print evaluation result
print(f"Test loss: {eval_result[0]}, Test accuracy: {eval_result[1]}")

# Save the model
model.save_pretrained('distilbert-finetuned-model')
tokenizer.save_pretrained('distilbert-finetuned-model')

# Optionally, load the model back
# model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-finetuned-model')
# tokenizer = DistilBertTokenizer.from_pretrained('distilbert-finetuned-model')

Epoch 1/3


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
5000/5000 [==============================] - 2017s 389ms/step - loss: 0.1516 - accuracy: 0.9266 - val_loss: 0.1069 - val_accuracy: 0.9373
Epoch 2/3
5000/5000 [==============================] - 1944s 389ms/step - loss: 0.0987 - accuracy: 0.9399 - val_loss: 0.0922 - val_accuracy: 0.9411
Epoch 3/3
 496/5000 [=>............................] - ETA: 27:17 - loss: 0.0914 - accuracy: 0.9431

In [ ]:
tokenizer.save_pretrained('distilbert-finetuned-model')

In [ ]:
import tensorflow as tf
from transformers import TFDistilBertForSequenceClassification, DistilBertTokenizer, DistilBertConfig
import matplotlib.pyplot as plt
import numpy as np

# Path where the fine-tuned model and tokenizer are saved
model_path = '/content/distilbert-finetuned-model'

# Load the tokenizer
tokenizer = DistilBertTokenizer.from_pretrained(model_path)

# Load the fine-tuned model with attention weights enabled
config = DistilBertConfig.from_pretrained(model_path, output_attentions=True)
model = TFDistilBertForSequenceClassification.from_pretrained(model_path, config=config)

# Mapping of labels to emotions
rating_to_emotion = {
    5: 'surprise',
    4: 'fear',
    3: 'angry',
    2: 'lovely',
    1: 'joy',
    0: 'sad'
}

# Function to predict emotion and get attention weights
def predict_emotion_with_attention(text):
    # Tokenize input text
    inputs = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=128,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='tf'
    )

    # Perform inference
    outputs = model(inputs)
    logits = outputs.logits
    probabilities = tf.nn.softmax(logits, axis=-1).numpy()[0]  # Convert logits to probabilities

    # Get predicted label
    predicted_label = tf.argmax(probabilities, axis=-1).numpy()

    # Get predicted emotion from the mapping
    predicted_emotion = rating_to_emotion[predicted_label]

    # Get attention weights
    attentions = outputs.attentions  # List of attention weights from each layer
    attention_weights = attentions[-1][0][0]  # Use the last layer's attention weights, for the first head

    return predicted_emotion, probabilities[predicted_label], attention_weights, inputs['input_ids']

# Example usage
text_input = "I feel really happy today!"
predicted_emotion, confidence, attention_weights, input_ids = predict_emotion_with_attention(text_input)

# Decode tokens
tokens = tokenizer.convert_ids_to_tokens(input_ids[0])

# Plot attention weights
def plot_attention(tokens, weights):
    fig, ax = plt.subplots(figsize=(10, 10))
    ax.matshow(weights, cmap='viridis')

    ax.set_xticks(range(len(tokens)))
    ax.set_yticks(range(len(tokens)))

    ax.set_xticklabels(tokens, rotation=90)
    ax.set_yticklabels(tokens)

    plt.show()

plot_attention(tokens, attention_weights.numpy())
print(f"Predicted Emotion: {predicted_emotion}")
print(f"Confidence: {confidence}")


In [12]:
import pandas as pd
import numpy as np
import requests
import tensorflow as tf
from transformers import TFDistilBertForSequenceClassification, DistilBertTokenizer, DistilBertConfig

# Path where the fine-tuned model and tokenizer are saved
model_path = '/content/drive/MyDrive/Colab Notebooks/model'

# Load the tokenizer
tokenizer = DistilBertTokenizer.from_pretrained(model_path)

# Load the fine-tuned model with attention weights enabled
config = DistilBertConfig.from_pretrained(model_path, output_attentions=True)
model = TFDistilBertForSequenceClassification.from_pretrained(model_path, config=config)

# Mapping of labels to emotions
rating_to_emotion = {
    5: 'surprise',
    4: 'fear',
    3: 'angry',
    2: 'lovely',
    1: 'joy',
    0: 'sad'
}

# Function to predict emotion and get attention weights
def predict_emotion_with_attention(text):
    # Tokenize input text
    inputs = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=128,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='tf'
    )

    # Perform inference
    outputs = model(inputs)
    logits = outputs.logits
    probabilities = tf.nn.softmax(logits, axis=-1).numpy()[0]  # Convert logits to probabilities

    # Get predicted label
    predicted_label = tf.argmax(probabilities, axis=-1).numpy()

    # Get predicted emotion from the mapping
    predicted_emotion = rating_to_emotion[predicted_label]

    # Get attention weights
    attentions = outputs.attentions  # List of attention weights from each layer
    attention_weights = attentions[-1][0][0]  # Use the last layer's attention weights, for the first head

    return predicted_emotion, probabilities[predicted_label], attention_weights, inputs['input_ids']

# Function to fetch top movies from TMDb with additional details
def get_top_movies_tmdb(genres, api_key, language, count):
    genre_ids = {
        'Action': 28,
        'Adventure': 12,
        'Animation': 16,
        'Biography': 36,  # TMDb does not have 'Biography', but using 'History' as closest alternative
        'Comedy': 35,
        'Crime': 80,
        'Drama': 18,
        'Family': 10751,
        'Fantasy': 14,
        'History': 36,
        'Horror': 27,
        'Musical': 10402,
        'Mystery': 9648,
        'Romance': 10749,
        'Sci-Fi': 878,
        'Thriller': 53,
        'War': 10752,
        'Western': 37
    }

    top_movies = []
    for genre in genres:
        genre_id = genre_ids.get(genre)
        if genre_id is not None:
            response = requests.get(f"https://api.themoviedb.org/3/discover/movie?api_key={api_key}&with_genres={genre_id}&with_original_language={language}&sort_by=vote_average.desc")
            data = response.json()
            if 'results' in data:
                for movie in data['results']:
                    movie_id = movie['id']
                    # Fetch movie details
                    movie_details = requests.get(f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={api_key}&append_to_response=credits,videos").json()
                    cast = [cast_member['name'] for cast_member in movie_details['credits']['cast'][:5]]  # Get top 5 cast members
                    trailer_link = None
                    for video in movie_details['videos']['results']:
                        if video['type'] == 'Trailer' and video['site'] == 'YouTube':
                            trailer_link = f"https://www.youtube.com/watch?v={video['key']}"
                            break
                    duration = movie_details.get('runtime', 'N/A')
                    top_movies.append({
                        'title': movie['title'],
                        'description': movie.get('overview', 'No description available.'),
                        'rating': movie.get('vote_average', 'N/A'),
                        'duration': duration,
                        'cast': ', '.join(cast),
                        'trailer_link': trailer_link
                    })
                    if len(top_movies) >= count:
                        break
        if len(top_movies) >= count:
            break

    # Remove duplicates
    seen_titles = set()
    unique_movies = []
    for movie in top_movies:
        if movie['title'] not in seen_titles:
            unique_movies.append(movie)
            seen_titles.add(movie['title'])

    return unique_movies[:count]

# Function to recommend movies based on text input
def recommend_movies_based_on_text(input_text, api_key):
    # Predict emotion and get attention weights
    predicted_emotion, confidence, attention_weights, input_ids = predict_emotion_with_attention(input_text)

    # Define a mapping from emotions to genres
    emotion_to_genres = {
        'surprise': ['Adventure', 'Fantasy'],
        'fear': ['Horror', 'Thriller'],
        'angry': ['Action', 'Crime'],
        'lovely': ['Romance', 'Family'],
        'joy': ['Comedy', 'Family'],
        'sad': ['Drama', 'Biography']
    }
    genres = emotion_to_genres.get(predicted_emotion, [])

    if not genres:
        return "No genres found for the given emotion."

    # Fetch top 5 Hindi movies for the genres
    top_hindi_movies = get_top_movies_tmdb(genres, api_key, 'hi', 5)
    # Fetch top 5 English movies for the genres
    top_english_movies = get_top_movies_tmdb(genres, api_key, 'en', 5)

    # Combine and return movies
    top_movies = top_hindi_movies + top_english_movies
    return top_movies

# Example usage
input_text = "I am feeling so happy today!"
api_key = '87bf218cee28571062421b51098698f5'
recommended_movies = recommend_movies_based_on_text(input_text, api_key)

# Create a DataFrame from the recommended movies
recommended_movies_df = pd.DataFrame(recommended_movies)

recommended_movies_df


Some layers from the model checkpoint at /content/drive/MyDrive/Colab Notebooks/model were not used when initializing TFDistilBertForSequenceClassification: ['dropout_79']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at /content/drive/MyDrive/Colab Notebooks/model and are newly initialized: ['dropout_159']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


,title,description,rating,duration,cast,trailer_link
0,Love and Cocoa,"It's the story of a beautiful silence of love,...",10.0,4,"Malharrao, Chahat Aggarwal",None
1,Angithee 2,"Kavya, a married woman, lives a dull life with...",10.0,0,Shafaq Naaz,https://www.youtube.com/watch?v=VlMrQ1DezJg
2,Pyar Mein Thoda Twist,Pyar me thoda twist is a Romcom and action dra...,10.0,226,"Mukesh J Bharti, Richa Mukherjee, Rajesh Sharm...",None
3,Sanwri - Love Beyond Gender,"Bhanwar, a simpleton young man in the rural Ra...",10.0,110,"Bhagwan Tiwari, Kanak Garg, Sarthak Narula, Gi...",None
4,Bekhudi,"A rich boy, Abhishek, falls for Sanya, who has...",10.0,122,"Adhyayan Suman, Anurag Sharma, Dev Sharma, Gul...",None
5,Fleeting Sunshine,When 2 women take a drug that makes them forge...,10.0,8,"Niamh O'Farrell Tyler, Lisa Bussi",None
6,A Pinch of Ginger,"Ginger has just moved to uni, but there's a pr...",10.0,15,"Carys Farndon, Tyler Thorpe, Lauren Assiter, M...",None
7,Royal Façade,Princess Avali flees her early coronation to U...,10.0,95,"Isaac Akers, Weixin Le, BJ Whimpey, Charla Boc...",https://www.youtube.com/watch?v=PY9cMIdT2ug
8,The Yearning,Two lonely people notice each other. A yearni...,10.0,6,"Thomas Eremia, Thomas Noon",https://www.youtube.com/watch?v=-05k4-ZSb_w
9,Luna Azul,"Two Venezuelans, Laura a catholic novice nun, ...",10.0,17,"Alan Corvaia, Ariana Gonzalez, Cesar Di Bello,...",None
